# 2nd Level Model Structure: Reboiler

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.energy_reboiler import energy_block_rule
from physics.VLE_reboiler_MPCC_P import VLE_block_rule
from physics.MPCC_P import P_NCP_block_rule, P_Reg_block_rule, P_pf_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,350+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.Q_main = pe.Var(within=pe.Reals) # MW

# Construct Individual Blocks

In [5]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [6]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.P_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [7]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [8]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

# MPCC - P-Pspec

In [9]:
model.MPCC = pe.Block(rule = P_NCP_block_rule)

> Importing MPCC_P_NCP Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.s_L
| MPCC.s_V
| MPCC.epi
--------------------------------------------------
> Adding complementarity constraint, spliting pressure used in VLE



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [10]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [11]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [12]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load a flash liquid composition as feed

In [13]:
with open('../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    C200 = pickle.load(f)

## Fixing In/Product Redundent Flow

In [14]:
model.y_.fix(0)
for i in m.COMP_TOTAL:
    model.x_['in',i].fix(C200.Solution.Variable['x[{}]'.format(i)]['Value'])

# model.L['in'].fix(1)
model.L['in'].setlb(1e-9)
# model.L['in'].setub(1e-8)

model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_V_.fix(0)
model.H_L_.fix(C200.Solution.Variable['H_L']['Value'])

## Fixing Model Parameters

In [15]:
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(0)
model.z.fix(0)
model.VLE_block.n_ave.fix(20)
# model.Q_main.fix(0)

# kappa

In [16]:
for i in model.component_objects(pe.Constraint, active=True):
    print(i.name)

mass_balance_main_con
VL_equil_con
summation_x_y_con
summation_total_mass_con
heat_balance_main_con
energy_block.dH_V_con
energy_block.H_V_con
energy_block.dH_vap_con
energy_block.dH_L_con
energy_block.H_L_con
energy_block.dH_F_con
energy_block.H_F_con
VLE_block.f_L_HENRY_con
VLE_block.Hen_con
VLE_block.Hen0_con
VLE_block.Hen_ref_con
VLE_block.Hen0_ref_con
VLE_block.f_L_NONHENRY_con
VLE_block.gamma_con
VLE_block.gamma_ref_con
VLE_block.n_ave_con
VLE_block.P_sat_con
VLE_block.P_sat_con2
VLE_block.P_sat_dY_inf_con
VLE_block.P_sat_dY0_con
VLE_block.f_V_con
VLE_block.V_L_nonHen_con
VLE_block.V_L_dY_inf_con
VLE_block.V_L_dY0_con
VLE_block.V_L_Hen_con
VLE_block.V_L_H2O_con
VLE_block.poynting_con
MPCC.s_L_complementarity_con
MPCC.s_V_complementarity_con
MPCC.pressure_equal_con


In [17]:
model.big_beta = pe.Var(within=pe.NonNegativeReals,initialize=1)
model.big_epi = pe.Param(initialize=1e-5,mutable = True)

In [18]:
all_expression = []
for i in model.component_data_objects(pe.Constraint, active=True):
    if i.name.startswith('summation_x_y_con') or i.name.startswith('mass_balance_main_con'):
        all_expression.append(i.body)
        i.deactivate()

In [19]:
model.test_list = pe.ConstraintList()
for i in all_expression:
    model.test_list.add(expr = i*model.big_beta==0);

In [20]:
model.complementarity_con = pe.Constraint(expr = 2*model.big_beta == \
                            model.L['in'] - model.big_epi + ((model.big_epi - model.L['in'])**2+1e-10)**0.5)
model.complementarity_con = pe.Constraint(expr = model.big_beta == model.L['in'])

	This is usually indicative of a modelling error.
	To avoid this warning, use block.del_component() and block.add_component().


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [21]:
model.T.fix(350+273.15)

In [22]:
check_DOF(pe,model)

Active Equality Constraints:	 960
Active Inequality Constraints:	 0
Active Variables:		 1132
Fixed Variables:		 171
DOF:				 1


In [23]:
# model.obj = pe.Objective(expr = - model.L['in'] - model.MPCC.pf ,sense=pe.maximize)
model.obj = pe.Objective(expr = - model.L['in'] ,sense=pe.maximize)

In [24]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  76  6.5170916e-06 1.43e-01 8.14e+02  -8.6 2.68e+02    -  3.91e-02 1.51e-01h  1
  77  3.0254404e-06 2.34e-02 1.34e+03  -8.6 2.51e+01    -  1.26e-01 1.00e+00h  1
  78  2.1093239e-06 1.99e-03 1.71e+01  -8.6 1.96e+01    -  4.14e-01 1.00e+00h  1
  79  1.2576191e-06 5.60e-03 3.52e+01  -8.6 1.29e+02    -  2.52e-01 4.54e-01H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  1.3557302e-06 5.51e-03 3.43e+01  -8.6 3.16e+02    -  6.48e-02 6.06e-02f  1
  81  1.4552173e-06 5.68e-03 5.34e+01  -8.6 2.55e+03    -  5.03e-03 5.14e-03f  2
  82  1.3566773e-06 5.83e-03 5.13e+01  -8.6 2.34e+03    -  9.95e-03 4.50e-03h  2
  83  1.2580446e-06 6.01e-03 4.24e+01  -8.6 1.40e+04    -  9.04e-04 9.04e-04s 12
  84r 1.2580446e-06 6.01e-03 9.99e+02  -2.2 0.00e+00    -  0.00e+00 0.00e+00R  1
  85r 3.6909259e-06 6.00e-03 9.99e+02  -2.2 1.43e+03    -  1.53e-07 4.26e-07f  9
  86r 1.2819932e-06 5.95e-03 9.98e+02  -2.2 2.97e+02    -  1.01e-03 8.17e-06f  1
  87r 5.2607376e-06 1.24e-03

 178  1.9271944e-08 2.75e-03 7.94e+04  -8.6 3.90e+02    -  1.00e+00 3.83e-02h  5
 179  1.9237698e-08 2.71e-03 7.65e+04  -8.6 4.69e+02    -  1.00e+00 4.33e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180  1.9162203e-08 2.65e-03 7.35e+04  -8.6 5.31e+02    -  1.00e+00 4.81e-02h  5
 181  1.9060959e-08 2.57e-03 7.06e+04  -8.6 5.62e+02    -  1.00e+00 5.14e-02h  5
 182  1.8836589e-08 2.54e-03 6.66e+04  -8.6 5.71e+02    -  1.00e+00 1.06e-01h  4
 183  1.7905369e-08 1.35e-02 1.57e+05  -8.6 4.72e+02    -  1.00e+00 7.74e-01w  1
 184  1.7131266e-08 1.38e-03 7.83e+04  -8.6 2.82e+02    -  1.00e+00 1.00e+00w  1
 185  2.0075660e-08 1.63e-05 5.05e+04  -8.6 4.56e+02    -  1.00e+00 1.00e+00w  1
 186  1.8720186e-08 2.49e-03 6.24e+04  -8.6 1.31e+03    -  1.00e+00 9.67e-02h  3
 187  1.8595397e-08 2.42e-03 5.84e+04  -8.6 4.62e+02    -  1.00e+00 9.98e-02h  4
 188  1.8464795e-08 2.32e-03 5.46e+04  -8.6 4.50e+02    -  1.00e+00 1.03e-01h  4
 189  1.8456660e-08 2.30e-03

In [25]:
model.Q_main.value

5.078551930369931e-07

In [26]:
model.big_beta.value

3.0534387213443954e-09

In [27]:
model.L['in'].value

3.0534387213443954e-09

In [28]:
model.T.value

623.15

In [29]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reboiler_MPCC_P_pf_200C.pickle','wb') as f:
#     pickle.dump(results,f)

In [30]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

Component		Liquid: 0.0000			Vapor: 0.0000			log K
------------------------------------------------------------------------------------------------------------
H2         		 0.0052%			 1.7648%			2.5345
CO         		 0.0086%			 2.6753%			2.4913
CO2        		 0.0367%			 7.4226%			2.3058
H2O        		 0.0459%			 7.1687%			2.1935
C2H4       		 0.0002%			 0.0345%			2.2248
C3H6       		 0.0009%			 0.0881%			2.0055
C4H8       		 0.0007%			 0.0905%			2.0857
C5H10      		 0.0021%			 0.1424%			1.8277
C6H12      		 0.0059%			 0.2469%			1.6214
C7H14      		 0.0143%			 0.4018%			1.4478
C8H16      		 0.0314%			 0.6222%			1.2969
C9H18      		 0.0634%			 0.9218%			1.1624
C10H20     		 0.1190%			 1.3067%			1.0405
C11H22     		 0.1328%			 1.1258%			0.9283
C12H24     		 0.2168%			 1.4455%			0.8240
C13H26     		 0.3296%			 1.7546%			0.7262
C14H28     		 0.4672%			 2.0099%			0.6337
C15H30     		 0.6196%			 2.1766%			0.5457
C16H32     		 0.7741%			 2.2404%			0.4615
C17H34     		 0.9194%			 2.2091%			0.3807
C

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = [];
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = [];
    
Trange = np.arange(350+273.15,199+273.15,-5)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {:.2f} K\t|\tV = {:.5f}\t|\tL = {:.5f}\t|'.format(model.T.value,model.V['out'].value,model.L['out'].value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)

# VLE Validation using AspenPlus

### Feed (Reactor Effluent)

In [ ]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.T.value-273.15))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.L['out'].value+model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.x[i].value*model.L['out'].value\
        +model.y[i].value*model.V['out'].value)/(model.L['out'].value+model.V['out'].value)))

### Phase Separation

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

In [ ]:
list(zip(rf_data['T'],rf_data['Q']))